# Projeto de Aprendizagem Automática II

## Procura de Exoplanetas no Espaço através da Emissão de Luz de Estrelas

### Importação de Bibliotecas

In [1]:
import pandas as pd 
import numpy as np  

import matplotlib            
import matplotlib.pyplot as plt

import seaborn as sns        
color = sns.color_palette()
sns.set_style('darkgrid')

import os

from random import randint
from scipy.stats import randint as sp_randint

from time import time
from datetime import datetime

import sklearn               
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, VarianceThreshold, chi2

from warnings import simplefilter

%matplotlib inline
simplefilter(action='ignore', category=FutureWarning)

### Carregamento dos Dados

O primeiro passo é o carregamento dos conjuntos de dados presentes em dois ficheiros distintos.

In [2]:
treino = pd.read_csv("../../../../Dados/dados_treino.csv")
teste = pd.read_csv("../../../../Dados/dados_teste.csv")

Em seguida, deve ser efetuada a divisão entre variáveis de decisão e atributos, permitindo o treino do modelo e teste dos vários registos.

In [3]:
X_train = treino.loc[:, treino.columns != 'LABEL'].values
Y_train = treino.LABEL.values

X_test = teste.loc[:, teste.columns != 'LABEL'].values
Y_test = teste.LABEL.values

### Modelo

Tendo em conta a existência de 2 classes distintas, o número de *clusters* do modelo a treinar deve ser, também, 2. Assim, tirando proveito da biblioteca *sklearn*, é possível gerar um modelo de *KMeans Clustering* para o problema em estudo.  

In [4]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=2)

Tendo o modelo definido, é necessário treinar o mesmo. Como se trata de um algoritmo não supervisionado, este apenas recebe as *features* dos dados.

In [5]:
model.fit(X_train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

### Teste

De modo a testar o modelo, a primeira ideia passou por comparar as *labels* geradas com as *labels* originais no conjunto de treino. Assim, como se pode ver em seguida, o modelo atribuiu os dados de forma próxima dos 50% para cada *cluster*, proporções bastante diferentes do real.

In [13]:
train_preds = model.predict(X_train)
print('Labels reais de valor 1:', sum(Y_train == 1))
print('Labels reais de valor 2:',sum(Y_train == 2))
print('Cluster de valor 0:', sum(train_preds == 0))
print('Cluster de valor 1:', sum(train_preds == 1))

Labels reais de valor 1: 5050
Labels reais de valor 2: 1515
Cluster de valor 0: 3360
Cluster de valor 1: 3205


Apesar destas proporções estarem bastante distantes do pretendido, vale sempre a pena verificar o que acontece com o conjunto de teste. Como se pode ver em seguida, o modelo classificou todos os sistemas como sendo pertencentes ao *cluster* 0, ou seja, este *cluster* representa a classe maioritária do problema.

In [14]:
y_pred = model.predict(X_test)
print('Labels reais de valor 1:', sum(Y_test == 1))
print('Labels reais de valor 2:',sum(Y_test == 2))
print('Cluster de valor 0:', sum(y_pred == 0))
print('Cluster de valor 1:', sum(y_pred == 1))

Labels reais de valor 1: 565
Labels reais de valor 2: 5
Cluster de valor 0: 570
Cluster de valor 1: 0


Em jeito de conclusão, para o problema em estudo, este algoritmo não é o mais apropriado. Isto deve-se ao facto de este não comparar valores vizinhos, ou seja, se duas séries temporais tiveram o mesmo valor de ponto máximo, mas em momentos distintos do tempo, eles não serão considerados como semelhantes. Assim, é necessária alguma tolerância temporal, podendo esta vir da utilização de algortimos apropriados ou do uso de métricas de *DTW* para comparar os registos.